<a href="https://colab.research.google.com/github/muhiqbalalamin/DeepLearning/blob/main/Regression_Second_Hand_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [3]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Folder DL/secondhanddataset.csv')

# Features and target
X = df.drop(columns=['current price', 'v.id'])
y = df['current price']

# Normalize target variable
y = (y - y.min()) / (y.max() - y.min())

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorch Model
class MLP_PyTorch(nn.Module):
    def __init__(self):
        super(MLP_PyTorch, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(X_train.shape[1], 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

# Training PyTorch model
pytorch_model = MLP_PyTorch()
criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.0005)

X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

for epoch in range(200):
    optimizer.zero_grad()
    outputs = pytorch_model(X_train_torch)
    loss = criterion(outputs, y_train_torch)
    loss.backward()
    optimizer.step()

# Predict and evaluate PyTorch model
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)
predictions = pytorch_model(X_test_torch).detach().numpy()

mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)

print(f"PyTorch MSE: {mse}")
print(f"PyTorch RMSE: {rmse}")
print(f"PyTorch R-squared: {r2}")

# TensorFlow Model
tf_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile and train TensorFlow model
tf_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), loss='mse')
tf_model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0)

# Predict and evaluate TensorFlow model
tf_predictions = tf_model.predict(X_test)

mse_tf = mean_squared_error(y_test, tf_predictions)
rmse_tf = np.sqrt(mse_tf)
r2_tf = r2_score(y_test, tf_predictions)

print(f"TensorFlow MSE: {mse_tf}")
print(f"TensorFlow RMSE: {rmse_tf}")
print(f"TensorFlow R-squared: {r2_tf}")

# Sample prediction
sample_idx = 0
sample_actual = y_test.iloc[sample_idx]
sample_features = X_test[sample_idx].reshape(1, -1)

pytorch_sample_pred = pytorch_model(torch.tensor(sample_features, dtype=torch.float32)).detach().numpy()[0][0]
tf_sample_pred = tf_model.predict(sample_features)[0][0]

print("--- Making Predictions with Trained Models ---")
print(f"Sample actual value: {sample_actual:.4f}")
print(f"PyTorch prediction: {pytorch_sample_pred:.4f}")
print(f"TensorFlow prediction: {tf_sample_pred:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Folder DL/secondhanddataset.csv'

# Penjelasan metrik evaluasi:
## MSE (Mean Squared Error):
MSE = (1/n) * sum((y_actual - y_predicted)^2), Semakin kecil MSE, semakin baik modelnya.

## RMSE (Root Mean Squared Error):
RMSE = sqrt(MSE), Memberikan interpretasi kesalahan dalam skala yang sama dengan target aslinya.

## R-Squared (R² Score):
R² = 1 - (SS_res / SS_tot), Dimana SS_res adalah jumlah kuadrat residual dan SS_tot adalah jumlah kuadrat total. Nilai R² yang lebih tinggi menunjukkan model yang lebih baik.